# Train/Validation/Test Rigor + Regression Metrics + Baseline Modeling

<hr>

<center>
<div>
<img src="https://raw.githubusercontent.com/davi-moreira/2026Summer_predictive_analytics_purdue_MGMT474/main/notebooks/figures/mgmt_474_ai_logo_02-modified.png" width="200"/>
</div>
</center>

# <center><a class="tocSkip"></center>
# <center>MGMT47400 Predictive Analytics</center>
# <center>Professor: Davi Moreira </center>

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/davi-moreira/2026Summer_predictive_analytics_purdue_MGMT474/blob/main/notebooks/03_regression_metrics_baselines.ipynb)

---

## Learning Objectives

By the end of this notebook, you will be able to:

1. Choose regression metrics aligned to business loss (MAE vs RMSE)
2. Establish a baseline model and interpret it correctly
3. Run holdout evaluation without contaminating the test set
4. Use quick diagnostic plots to spot obvious modeling issues
5. Document evaluation decisions (metric, split, baseline, assumptions)

---

> **📋 Participation Reminder:** This notebook contains **2 PAUSE-AND-DO exercises**. You are expected to complete all exercises before submitting your notebook.

---

## 1. Setup

Before we can measure model quality, we need the standard scientific-Python stack: **pandas** for tabular data, **NumPy** for numerics, **matplotlib/seaborn** for plotting, and several **scikit-learn** modules for splitting, scoring, and modeling.
All random operations use `RANDOM_SEED = 474` so every student sees identical results.
Run the cell below once; the checkmark confirms every library loaded successfully.

In [ ]:
# Core imports
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.datasets import fetch_california_housing
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
from sklearn.dummy import DummyRegressor
from sklearn.linear_model import LinearRegression
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
import warnings

warnings.filterwarnings('ignore')
pd.set_option('display.precision', 4)
sns.set_style('whitegrid')
plt.rcParams['figure.figsize'] = (10, 6)

RANDOM_SEED = 474
np.random.seed(RANDOM_SEED)

print("✓ Setup complete!")

**Reading the output:**

The single line `Setup complete!` confirms that all imports succeeded and the environment is ready. Behind the scenes, the cell loaded **pandas**, **NumPy**, **matplotlib**, **seaborn**, and five scikit-learn modules (`train_test_split`, three metric functions, `DummyRegressor`, `LinearRegression`, `Pipeline`, and `StandardScaler`). The global random seed is set to **RANDOM_SEED = 474**, which controls every random operation in this notebook so that your numbers match the solutions exactly.

**Why this matters:** If any import had failed, you would see a `ModuleNotFoundError` here instead of the checkmark. Catching import problems at the top of the notebook prevents cryptic errors later.

---

## 2. Load Data and Create Splits

We use the California Housing dataset, which contains **20,640 census-tract records** with 8 numeric features (median income, average rooms, latitude, etc.) and a continuous target: **median house value** in units of $100,000.
The data is split 60/20/20 into train, validation, and test sets using our course-standard `RANDOM_SEED = 474`.
The test set is immediately locked away; all model development proceeds on train + validation only.

> 💡 **Gemini Prompt:** "Load the California Housing dataset from scikit-learn as a DataFrame. Separate the target column 'MedHouseVal' from the features. Then split the data into train (60%), validation (20%), and test (20%) sets using train_test_split with random_state=474. Print the sizes of each split."
>
> **After running, verify:**
> - Train set is ~60% of total samples (about 12,384 rows)
> - Validation and test sets are each ~20% (about 4,128 rows)
> - The target variable `y` contains median house values


In [ ]:
# Load dataset
california = fetch_california_housing(as_frame=True)
df = california.frame

X = df.drop(columns=['MedHouseVal'])
y = df['MedHouseVal']

# Create splits
X_temp, X_test, y_temp, y_test = train_test_split(X, y, test_size=0.20, random_state=RANDOM_SEED)
X_train, X_val, y_train, y_val = train_test_split(X_temp, y_temp, test_size=0.25, random_state=RANDOM_SEED)

print(f"Train: {len(X_train)} | Validation: {len(X_val)} | Test: {len(X_test)} (LOCKED)")
print(f"\n⚠️ TEST SET IS LOCKED - Do not use until final evaluation!")

**Reading the output:**

The split summary shows three partition sizes that sum to **20,640** total samples: roughly **12,384 training**, **4,128 validation**, and **4,128 test** records (the exact 60/20/20 ratio). The warning `TEST SET IS LOCKED` is intentional: it reminds you that the test partition must remain untouched until the very last evaluation. Any intermediate peeking at test performance would leak information into your model-selection decisions and inflate your reported results.

**Key takeaway:** Memorize the counts. When you later compute metrics and something looks wrong, come back here to verify you are evaluating on the correct set (validation, not test).

---

## 3. Regression Metrics: MAE, RMSE, R²

### 3.1 Understanding the Metrics

**Mean Absolute Error (MAE)**
- Average of absolute differences
- Same units as target
- Less sensitive to outliers
- Use when: All errors cost the same

**Root Mean Squared Error (RMSE)**
- Square root of average squared differences
- Same units as target
- More sensitive to outliers (penalizes large errors)
- Use when: Large errors are disproportionately costly

**R² (Coefficient of Determination)**
- Proportion of variance explained
- Scale: 0 (no better than mean) to 1 (perfect)
- Can be negative if model is worse than mean
- Use when: You want a relative improvement metric

> 💡 **Gemini Prompt:** "Write a Python function called `evaluate_regression` that takes `y_true`, `y_pred`, and an optional `name` parameter. It should compute MAE (mean absolute error), RMSE (root mean squared error), and R-squared. Return the results as a dictionary with keys 'Model', 'MAE', 'RMSE', and 'R²'."
>
> **After running, verify:**
> - Function returns a dictionary with exactly 4 keys: 'Model', 'MAE', 'RMSE', 'R²'
> - RMSE is computed as the square root of mean_squared_error (not MSE itself)
> - Function uses scikit-learn's `mean_absolute_error`, `mean_squared_error`, and `r2_score`


In [ ]:
def evaluate_regression(y_true, y_pred, name="Model"):
    """
    Compute standard regression metrics.
    
    Parameters:
    -----------
    y_true : array-like
        True target values
    y_pred : array-like
        Predicted values
    name : str
        Name for reporting
    
    Returns:
    --------
    dict : Dictionary of metrics
    """
    mae = mean_absolute_error(y_true, y_pred)
    rmse = np.sqrt(mean_squared_error(y_true, y_pred))
    r2 = r2_score(y_true, y_pred)
    
    metrics = {
        'Model': name,
        'MAE': mae,
        'RMSE': rmse,
        'R²': r2
    }
    
    return metrics

print("✓ Evaluation function created")

**Reading the output:**

The message `Evaluation function created` tells you the helper `evaluate_regression()` is now available. This function accepts true values, predicted values, and a label, then returns a dictionary with three metrics: **MAE** (mean absolute error), **RMSE** (root mean squared error), and **R²** (coefficient of determination). No predictions have been made yet; the function simply lives in memory, ready to score any model you hand it.

**Why this matters:** Centralizing metric computation in one function prevents copy-paste errors. Every model in this notebook will be scored through the same function, guaranteeing consistent comparisons.

---

## 4. Baseline Models

### 4.1 Why Baselines Matter

A baseline model provides a reference point. Any model should beat these simple strategies:
- **Mean baseline**: Predict the training set mean for all samples
- **Median baseline**: Predict the training set median for all samples

If your model doesn't beat the baseline, something is wrong.

> 💡 **Gemini Prompt:** "Create two baseline models using scikit-learn's `DummyRegressor`: one with strategy='mean' and one with strategy='median'. Fit each on the training data, then generate predictions for both the training and validation sets."
>
> **After running, verify:**
> - Both `DummyRegressor` models are fitted on `X_train` and `y_train`
> - Predictions are generated for both train and validation sets (4 prediction arrays total)
> - Mean baseline predictions should all be the same value (the training set mean)


In [ ]:
# Mean baseline
baseline_mean = DummyRegressor(strategy='mean')
baseline_mean.fit(X_train, y_train)
y_pred_mean_train = baseline_mean.predict(X_train)
y_pred_mean_val = baseline_mean.predict(X_val)

# Median baseline
baseline_median = DummyRegressor(strategy='median')
baseline_median.fit(X_train, y_train)
y_pred_median_train = baseline_median.predict(X_train)
y_pred_median_val = baseline_median.predict(X_val)

print("✓ Baseline models fitted")

**Reading the output:**

The message `Baseline models fitted` confirms that two `DummyRegressor` objects are now trained: one with `strategy='mean'` and one with `strategy='median'`. Despite sounding trivial, fitting a dummy regressor is necessary so that scikit-learn stores the learned constant (the training-set mean or median) and can later call `.predict()` consistently. These baselines represent the floor of performance: any real model must beat them.

**Key takeaway:** Baselines require almost no computation, yet they give you the single most important reference number in your evaluation. Always establish a baseline before building complex models.

---

## 📝 PAUSE-AND-DO Exercise 1 (5 minutes)

**Task:** Write `evaluate_regression(y_true, y_pred)` returning MAE/RMSE/R².

The function is already implemented above. Now:
1. Use it to evaluate both baselines on train and validation sets
2. Create a comparison table
3. Interpret the results

---

> 💡 **Gemini Prompt:** "Evaluate both baseline models (mean and median) on training and validation sets using the `evaluate_regression` function. Collect results into a list, convert to a DataFrame, and print a comparison table. Also print the MAE interpretation in dollar terms (values are in $100,000s)."
>
> **After running, verify:**
> - Results DataFrame has 4 rows: Mean Baseline (Train), Mean Baseline (Val), Median Baseline (Train), Median Baseline (Val)
> - R² is 0.0 (or near 0) for the mean baseline on training data
> - MAE values are displayed and interpreted in context (multiply by $100,000)


In [ ]:
# Evaluate baselines
results = []

# Mean baseline - train
results.append(evaluate_regression(y_train, y_pred_mean_train, "Mean Baseline (Train)"))
# Mean baseline - validation
results.append(evaluate_regression(y_val, y_pred_mean_val, "Mean Baseline (Val)"))

# Median baseline - train
results.append(evaluate_regression(y_train, y_pred_median_train, "Median Baseline (Train)"))
# Median baseline - validation
results.append(evaluate_regression(y_val, y_pred_median_val, "Median Baseline (Val)"))

results_df = pd.DataFrame(results)
print("=== BASELINE COMPARISON ===")
print(results_df)

print(f"\n💡 Key insights:")
print(f"  - R² = 0 means the model is no better than predicting the mean")
print(f"  - MAE = {results_df.loc[1, 'MAE']:.3f} means on average we're off by ${results_df.loc[1, 'MAE']:.1f}00,000")
print(f"  - RMSE > MAE indicates some large errors in predictions")

**Reading the output:**

The comparison table shows four rows: mean and median baselines, each scored on both train and validation sets. Notice that **R² = 0.00** (or very close) for both baselines. This is expected: by definition, predicting the mean explains zero variance. The **MAE** column tells you the average prediction error in the original units ($100k). For example, if MAE is roughly **0.91**, that means the mean-predictor is off by about **$91,000** per census tract on average. **RMSE** is larger than MAE, indicating that some predictions have especially large errors that the squared penalty amplifies.

**Why this matters:** These baseline numbers become your "bar to clear." When you evaluate linear regression next, you will compare its MAE and R² directly to these rows. Any model that cannot beat the mean predictor adds no value.

---

## 5. Simple Linear Model

Baselines tell us how well "doing nothing smart" performs. Now let's see if even the simplest machine-learning model, ordinary linear regression, can beat the guess-the-mean strategy.
We wrap a `StandardScaler` and `LinearRegression` inside a scikit-learn `Pipeline` so that scaling and fitting happen in a single reproducible step.
If the linear model cannot improve over the mean baseline, something fundamental is wrong with our features or our data.

> 💡 **Gemini Prompt:** "Create a scikit-learn Pipeline that first applies `StandardScaler` and then fits a `LinearRegression` model. Fit it on the training data and generate predictions for both training and validation sets."
>
> **After running, verify:**
> - Pipeline has two steps: 'scaler' (StandardScaler) and 'regressor' (LinearRegression)
> - Pipeline is fitted on `X_train` and `y_train` only
> - Predictions are generated for both train and validation sets


In [ ]:
# Create pipeline with scaling + linear regression
linear_pipeline = Pipeline([
    ('scaler', StandardScaler()),
    ('regressor', LinearRegression())
])

# Fit on training data
linear_pipeline.fit(X_train, y_train)

# Predict
y_pred_linear_train = linear_pipeline.predict(X_train)
y_pred_linear_val = linear_pipeline.predict(X_val)

print("✓ Linear model fitted")

**Reading the output:**

The confirmation `Linear model fitted` means scikit-learn has estimated coefficients for all 8 features inside a `Pipeline` that first applies `StandardScaler` (zero-mean, unit-variance) and then fits `LinearRegression`. No metrics are printed yet; this cell only fits the model and stores predictions. The scaling step is critical because it ensures that coefficient magnitudes are comparable across features with very different scales (e.g., latitude in degrees vs. income in tens of thousands).

**Key takeaway:** Wrapping preprocessing and modeling inside a single `Pipeline` prevents a common mistake called *fit-transform leakage*, where the scaler accidentally sees validation data during fitting.

---

## 📝 PAUSE-AND-DO Exercise 2 (5 minutes)

**Task:** Compare baseline vs linear regression and interpret the delta.

---

> 💡 **Gemini Prompt:** "Evaluate the linear regression model on train and validation sets using `evaluate_regression`, append to the existing results list, and create a full comparison DataFrame. Calculate the percentage improvement in MAE of linear regression over the mean baseline on the validation set."
>
> **After running, verify:**
> - Full results DataFrame has 6 rows (4 baseline + 2 linear regression)
> - Linear regression R² on validation is substantially above 0 (model learns patterns)
> - Improvement percentage is computed as `(baseline_MAE - linear_MAE) / baseline_MAE * 100`


In [ ]:
# Add linear model results
results.append(evaluate_regression(y_train, y_pred_linear_train, "Linear Regression (Train)"))
results.append(evaluate_regression(y_val, y_pred_linear_val, "Linear Regression (Val)"))

# Create full comparison table
full_results_df = pd.DataFrame(results)
print("=== FULL MODEL COMPARISON ===")
print(full_results_df)

# Calculate improvements
baseline_mae = full_results_df.loc[full_results_df['Model'] == 'Mean Baseline (Val)', 'MAE'].values[0]
linear_mae = full_results_df.loc[full_results_df['Model'] == 'Linear Regression (Val)', 'MAE'].values[0]
improvement_pct = ((baseline_mae - linear_mae) / baseline_mae) * 100

print(f"\n=== IMPROVEMENT OVER BASELINE ===")
print(f"Baseline MAE: {baseline_mae:.4f}")
print(f"Linear MAE: {linear_mae:.4f}")
print(f"Improvement: {improvement_pct:.2f}%")
print(f"\nLinear R² on validation: {full_results_df.loc[full_results_df['Model'] == 'Linear Regression (Val)', 'R²'].values[0]:.4f}")
print(f"This means the model explains {full_results_df.loc[full_results_df['Model'] == 'Linear Regression (Val)', 'R²'].values[0]*100:.2f}% of variance")

**Reading the output:**

The full comparison table now includes six rows: train and validation scores for the mean baseline, median baseline, and linear regression. The key numbers to focus on are the validation columns. Linear regression should show a dramatically lower **MAE** and a positive **R²** (likely around **0.59--0.61**), whereas both baselines sit at R² near zero. The improvement percentage printed below the table quantifies the MAE drop; a figure around **30--40%** is typical for this dataset. Also compare **train vs. validation R²** for linear regression: a small gap (< 0.02) suggests minimal overfitting.

**Why this matters:** This table is the prototype for every model comparison you will build in this course. The pattern -- baselines first, then progressively stronger models, always checking validation -- is the core workflow of applied predictive analytics.

---

### YOUR INTERPRETATION HERE:

**Observation 1: Performance**  
[How much better is linear vs baseline?]

**Observation 2: Overfitting Check**  
[Compare train vs validation scores - is there overfitting?]

**Observation 3: Business Context**  
[What does this MAE mean in practical terms?]

---

## 6. Residual Analysis

Residuals = True values - Predicted values

Good residuals should:
- Be centered around zero
- Show no patterns (random scatter)
- Have roughly constant variance

> 💡 **Gemini Prompt:** "Calculate residuals (actual minus predicted) for the linear regression model on both training and validation sets. Create a 2x2 subplot figure: top row shows residuals vs predicted values (scatter with y=0 reference line) for train and validation; bottom row shows residual histograms for train and validation. Print residual mean and standard deviation."
>
> **After running, verify:**
> - Residuals are computed as `y_true - y_pred` (not the reverse)
> - All four subplots display correctly: two scatter plots (top) and two histograms (bottom)
> - Residual means are very close to zero (especially for the training set)


In [ ]:
# Calculate residuals
residuals_train = y_train - y_pred_linear_train
residuals_val = y_val - y_pred_linear_val

# Create residual plots
fig, axes = plt.subplots(2, 2, figsize=(14, 10))

# Plot 1: Residuals vs Predicted (Train)
axes[0, 0].scatter(y_pred_linear_train, residuals_train, alpha=0.5)
axes[0, 0].axhline(y=0, color='r', linestyle='--')
axes[0, 0].set_xlabel('Predicted Values')
axes[0, 0].set_ylabel('Residuals')
axes[0, 0].set_title('Residual Plot - Training Set')

# Plot 2: Residuals vs Predicted (Validation)
axes[0, 1].scatter(y_pred_linear_val, residuals_val, alpha=0.5, color='orange')
axes[0, 1].axhline(y=0, color='r', linestyle='--')
axes[0, 1].set_xlabel('Predicted Values')
axes[0, 1].set_ylabel('Residuals')
axes[0, 1].set_title('Residual Plot - Validation Set')

# Plot 3: Distribution of Residuals (Train)
axes[1, 0].hist(residuals_train, bins=50, alpha=0.7, edgecolor='black')
axes[1, 0].axvline(x=0, color='r', linestyle='--')
axes[1, 0].set_xlabel('Residual')
axes[1, 0].set_ylabel('Frequency')
axes[1, 0].set_title('Residual Distribution - Training')

# Plot 4: Distribution of Residuals (Validation)
axes[1, 1].hist(residuals_val, bins=50, alpha=0.7, color='orange', edgecolor='black')
axes[1, 1].axvline(x=0, color='r', linestyle='--')
axes[1, 1].set_xlabel('Residual')
axes[1, 1].set_ylabel('Frequency')
axes[1, 1].set_title('Residual Distribution - Validation')

plt.tight_layout()
plt.show()

print("=== RESIDUAL STATISTICS ===")
print(f"Training residuals: mean={residuals_train.mean():.6f}, std={residuals_train.std():.4f}")
print(f"Validation residuals: mean={residuals_val.mean():.6f}, std={residuals_val.std():.4f}")

**Reading the output:**

The 2x2 grid shows four panels. **Top row**: residuals (true minus predicted) plotted against predicted values for train (left) and validation (right). Ideally you want a formless cloud centered on the red zero-line; any visible pattern (funnel, curve, clustering) signals a problem. **Bottom row**: histograms of residuals for train and validation. A roughly bell-shaped distribution centered near zero is good; heavy right tails suggest the model under-predicts expensive properties. The printed statistics confirm that the mean residual is near zero (by construction for OLS) and report the standard deviation, which should approximately match the RMSE from earlier.

**Key takeaway:** If you see a clear funnel shape (variance increasing with predicted value), the model exhibits *heteroskedasticity*, meaning errors are not constant across the prediction range. This is common in housing data, where price variability is higher for expensive areas.

---

## 7. Predicted vs Actual Plot

A predicted-vs-actual scatter plot is the most intuitive diagnostic for a regression model. Every point represents one sample; the x-axis is the true value and the y-axis is the prediction.
Perfect predictions fall exactly on the red 45-degree diagonal. Systematic deviations from the line reveal regions where the model consistently over- or under-predicts.
We plot train and validation side by side to check whether the model generalizes.

> 💡 **Gemini Prompt:** "Create a side-by-side predicted vs actual scatter plot for the linear regression model. Plot training set on the left and validation set on the right. Add a red dashed 45-degree reference line (perfect predictions) on each subplot."
>
> **After running, verify:**
> - The 45-degree reference line goes from min to max of actual values
> - Points clustered near the red line indicate good predictions
> - Training and validation plots show similar scatter patterns (no severe overfitting)


In [ ]:
# Predicted vs Actual
fig, axes = plt.subplots(1, 2, figsize=(14, 5))

# Training set
axes[0].scatter(y_train, y_pred_linear_train, alpha=0.5)
axes[0].plot([y_train.min(), y_train.max()], [y_train.min(), y_train.max()], 'r--', lw=2)
axes[0].set_xlabel('Actual Values')
axes[0].set_ylabel('Predicted Values')
axes[0].set_title('Training Set: Predicted vs Actual')

# Validation set
axes[1].scatter(y_val, y_pred_linear_val, alpha=0.5, color='orange')
axes[1].plot([y_val.min(), y_val.max()], [y_val.min(), y_val.max()], 'r--', lw=2)
axes[1].set_xlabel('Actual Values')
axes[1].set_ylabel('Predicted Values')
axes[1].set_title('Validation Set: Predicted vs Actual')

plt.tight_layout()
plt.show()

print("💡 Points close to the red line indicate good predictions")
print("   Points far from the line show prediction errors")

**Reading the output:**

The two scatter plots compare true values (x-axis) to predictions (y-axis) for training and validation. Points hugging the red 45-degree line indicate accurate predictions. You will likely notice a horizontal "ceiling" near the maximum target value (~5.0, representing $500k), where the model clips because California Housing caps median values. Dispersion around the line is wider at higher actual values, consistent with the heteroskedasticity observed in the residual plots.

**Why this matters:** This plot is the fastest way to communicate model quality to a non-technical stakeholder. A tight cluster along the diagonal is immediately interpretable as "the model works." The visible ceiling effect also warns that the dataset has a truncation issue that no linear model can fully resolve.

---

## 8. Test Set Lockbox Discipline

### The Test Set Rule:

> **"Touch the test set ONCE, at the very end"**

**Why?**
- Every time you look at test performance, you risk making decisions based on it
- This causes "leakage" of test information into your modeling choices
- The test set must remain a true "unseen" evaluation

**What to do instead:**
- Use validation set for all model selection and tuning
- Use cross-validation for robust comparisons
- Only evaluate on test when you're completely done

**Warning signs you're peeking:**
- "Let me just check test performance real quick"
- "The test score is lower, let me adjust..."
- Running multiple experiments and looking at test each time

In [ ]:
print("=== TEST SET STATUS ===")
print(f"Test set size: {len(X_test)} samples")
print(f"Test set is LOCKED 🔒")
print(f"\n✓ We will NOT evaluate on test until the final submission")
print(f"✓ All model development uses only train + validation")

**Reading the output:**

The printout confirms that the test set contains **4,128 samples** and remains locked. No test-set metrics have been computed anywhere in this notebook. This discipline is central to honest evaluation: the test set is a one-time, final report card. If you evaluate on it repeatedly and adjust your model based on those numbers, you are effectively fitting to the test set and your reported performance will be optimistically biased.

**Key takeaway:** Treat the test set like a sealed envelope. You will open it once, at the very end of the modeling process, and report whatever number comes out -- good or bad.

---

## 9. Evaluation Documentation Template

Every evaluation should document:

### Evaluation Plan

**Primary Metric:** [MAE / RMSE / R²]  
**Rationale:** [Why this metric aligns with business goals]

**Split Strategy:**
- Training: 60% (for fitting)
- Validation: 20% (for selection)
- Test: 20% (for final evaluation)

**Baseline:** [Mean / Median predictor]  
**Baseline Performance:** [Validation score]

**Model:** [Linear Regression with Standard Scaling]  
**Model Performance:** [Validation score]  
**Improvement over Baseline:** [Percentage or absolute difference]

**Assumptions:**
- Features are available at prediction time
- Relationship is approximately linear
- No major data quality issues

**Risks:**
- Model may not generalize to different time periods
- Assumes no distribution shift
- Sensitive to outliers

## 10. Wrap-Up: Key Takeaways

### What We Learned Today:

1. **Metrics Matter**: Choose MAE/RMSE/R² based on business loss function
2. **Baselines First**: Always establish a simple baseline for comparison
3. **Holdout Discipline**: Train on train, evaluate on validation, lock test away
4. **Residuals Tell Stories**: Use diagnostic plots to spot issues
5. **Document Everything**: Clear evaluation plans prevent confusion later

### Critical Rules:

> **"If your model can't beat the mean, debug before proceeding"**

> **"The test set is a lockbox - open it once"**

### Next Steps:

- Next notebook: Linear regression with features and diagnostics
- We'll build on today's evaluation framework
- Start thinking about which metrics matter for your project

---

## Participation Assignment Submission Instructions

### To Submit This Notebook:

1. **Complete all exercises**: Fill in both PAUSE-AND-DO exercise cells with your findings
2. **Run All Cells**: Execute `Runtime → Run all` to ensure everything works
3. **Save a Copy**: `File → Save a copy in Drive or Download the .ipynb extension`
4. **Submit**: Upload your `.ipynb` file in the participation assignment you find in the course Brightspace page.

### Before Submitting, Check:

- [ ] All cells execute without errors
- [ ] All outputs are visible
- [ ] Both exercise responses are complete
- [ ] Notebook is shared with correct permissions
- [ ] You can explain every line of code you wrote

### Next Step:

Complete the **Quiz** in Brightspace (auto-graded)

---

## 11. Submission Instructions

### To Submit:

1. Run all cells
2. Complete both exercises
3. Write a 3-sentence evaluation note answering:
   - Which metric is most appropriate for this problem and why?
   - How much better is linear regression than the baseline?
   - What does the residual analysis suggest?
4. Submit Colab link + evaluation note in LMS

---

## Bibliography

- James, G., Witten, D., Hastie, T., & Tibshirani, R. (2021). *An Introduction to Statistical Learning with Python* - Chapter on Model Assessment and Selection
- Hastie, T., Tibshirani, R., & Friedman, J. (2009). *The Elements of Statistical Learning* - Test error, training error, bias-variance
- scikit-learn User Guide: [Regression metrics](https://scikit-learn.org/stable/modules/model_evaluation.html#regression-metrics)
- scikit-learn User Guide: [Common pitfalls](https://scikit-learn.org/stable/common_pitfalls.html)

---



<center>

Thank you!

</center>